In [143]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [144]:
%sendtofile main.py --source main.py

Sent 125 lines (2963 bytes) to main.py.


In [ ]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.envh")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

In [ ]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]
api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)
runner.run()

In [ ]:
print(runner.output_buffer)
runner.stop()

In [ ]:
f = open('lib/.envh', 'r')
print(f.read())

In [ ]:
%sendtofile main.py --source main.py

# Local functionality

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(calibration_data[7])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(calibration_data[10])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)
print(calibration_data)

In [ ]:
print(calibration_data)

# Local with downsample

In [45]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['b\'\\r\\nB\\x02ets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,len:3496\\r\\nentry 0x4008063c\\r\\n\\r\\nBroets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\\r\\nmode:DIO, clock div:2\\r\\nload:0x3fff0018,len:4\\r\\nload:0x3fff001c,len:5204\\r\\nload:0x40078000,len:12136\\r\\nload:0x40080400,len:3496\\r\\nentry 0x4008063c\\r\\n\\r\\nBro\\x17"\\x95\\xd1\\x95\\x8d\\xd1\\xbd\\xc9\\x81was triggered\\r\\n\\r\\nets Jul 29 2019 12:21:46\\r\\n\\r\\nrst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\\r\\nconfigsip: 0, SPIWP:0xee\\r\\nclk_dr

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(calibration_data[7])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(calibration_data[10])
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

decode = Runner('MsetCCA', buffer_size=256) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

In [ ]:
print(calibration_data[7][0])
print(calibration_data[7][1])
print(calibration_data[7][2])

In [ ]:
decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

print(4)

decode.run()

print(5)

In [ ]:
for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoded_output)
    p13.off()

print(6)
decode.stop()
print(7)

# Online Functionality

In [39]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

# env_vars = load_env_vars("lib/.envh")
# # connect WiFI
# ssid = env_vars.get("WIFI_SSID")
# password = env_vars.get("WIFI_PASSWORD")
# ssid ="Rishil"
# password = "rishilhotspot"
# connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"
log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='hotspot_test_wireless')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


In [ ]:
runner.stop()

In [ ]:
print(runner.decoded_output)

# Online with downsample

In [ ]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

print(3)

del runner

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration
api_host = "http://192.168.0.13:5001/"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='downsample_test')

runner = OnlineRunner('MsetCCA', buffer_size=Ns)
runner.setup(**log_params)

runner.calibrate(calibration_data)
runner.run()


In [ ]:
runner.calibrate(calibration_data)

In [ ]:
runner.stop()

In [ ]:
print(runner.decoder.decoder_stack[12].Y)

# Data send

In [ ]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

In [ ]:
%sendtofile main.py --source main.py

In [ ]:
%sendtofile lib/decoding.py --source lib/decoding.py

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

# counter = 0
# while counter !=10:
#     if (counter % 2 == 0):
#         p26.on()
#         p13.off()
#     else:
#         p13.on()
#         p26.off()
#     time.sleep(1)
#     counter+=1

p26.off()
p13.off()

time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

from lib.runner import OnlineRunner
from lib.logging import logger_types

#api_host = "http://172.20.10.2:5001/" # make sure the port corresponds to your logging server configuration

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 3

api_host = "http://192.168.0.13:5001/lol"

log_params = dict(server=api_host, 
                  log_period=4, 
                  logger_type=logger_types.HTTP, 
                  send_raw=True, 
                  session_id='ssvep')

runner = OnlineRunner('CCA', buffer_size=Ns)
runner.setup(**log_params)

runner.run()

In [ ]:
runner.stop()

In [ ]:
print(runner.output )

# sending calibration data with posts

Potentially could be the decoding on the server side not working, maybe bypass this by sending it array by array, but this didn;t work last time I tried.

In [83]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

serial exception on close write failed: [Errno 6] Device not configured
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


Listing directory 'lib'.
       55    lib/.env
        0    lib/__init__.py
             lib/__pycache__/
     4006    lib/computation.py
      446    lib/config.py
     5219    lib/decoding.py
      855    lib/diagnostics.py
     3503    lib/logging.py
     1254    lib/networking.py
     6445    lib/newumqtt.py
     5697    lib/peripherals.py
    27364    lib/requests.py
     7664    lib/runner.py
     1911    lib/scheduling.py
     2067    lib/signal.py
     6570    lib/umqtt.py
     1691    lib/utils.py
             lib/websocket/
      976    lib/websockets.py


In [76]:
from lib.utils import connect_wifi, load_env_vars

ssid = 'Aditya'
password = '123456789!'
connect_wifi(ssid, password)

connecting to network...
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
  File "lib/utils.py", line 44, in connect_wifi
KeyboardInterrupt: 


In [ ]:
def connect_wifi(ssid, password):
    import network

    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    if not wlan.isconnected():
        print("connecting to network...")
        wlan.connect(ssid, password)
        while not wlan.isconnected():  # okay that this is blocking
            pass
    print("network config:", wlan.ifconfig())
    return wlan

from machine import Pin

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)

p26.on()
p13.on()

ssid = 'Rishil'
password = 'rishilhotspot'
connect_wifi(ssid, password)

p26.off()
p13.off()

In [ ]:
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

In [ ]:
payload = {"b":[123]}#json.dumps({"a":[123]})
a = requests.JSONRequest("http://192.168.0.13:5001/", payload)
b = requests.JSONRequest("http://192.168.0.13:5001/calibration",payload)
print(a)
print(b)

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

# time.sleep(5)

# p26.off()
# p13.off()

# time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner  


def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return data

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s))
        
    runner.stop()

    gc.collect()
    p26.off()
    return trials

print(1)

decode_period_s = 4 # decode every x seconds
Ns = 256
Nt = 4
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup()

p26.off()
p13.off()

calibration_data = {}
print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

calibration_data[7] = getData(Nt, decode_period_s)
toSend = {"7": calibration_data[7]}
requests.JSONRequest("http://192.168.0.13:5001/7hz", toSend)
del calibration_data[7]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print("=====================================================")
toSend = {"10": calibration_data[10]}
requests.JSONRequest("http://192.168.0.13:5001/10hz", toSend)
del calibration_data[10]
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
toSend = {"12": calibration_data[12]}
requests.JSONRequest("http://192.168.0.13:5001/12hz", toSend)
del calibration_data
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
print(3)
runner.run()

while True:
    time.sleep(4)
    data = preprocess_data(runner.output_buffer)
    toSend = {"raw_data_downsampled":data}
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)

In [ ]:
x = requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
print(x)

In [ ]:
runner.stop()

In [ ]:
data = preprocess_data(runner.output_buffer)
print(data)
toSend = {"raw_data_downsampled":data}
requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)

In [ ]:
runner.stop()

In [ ]:
def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return signal[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = preprocess_data(runner.output_buffer)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

In [ ]:
requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
print(3)
runner.run()

while True:
    time.sleep(4)
    data = preprocess_data(runner.output_buffer)
    toSend = {"raw_data_downsampled":data}
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)

In [9]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars
from ulab import numpy as np
import utime as time
from lib.runner import Runner 

ssid = 'vodafone749282'
password = 'c4Eqssga2YAdAZ97'
connect_wifi(ssid, password)
#192.168.1.106

runner = Runner('CCA', buffer_size=Ns)
runner.setup()

runner.run()

while True:
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"data":data}
    print(toSend)
#     requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
    requests.JSONRequest("http://192.168.1.106:5001/", toSend)

network config: ('192.168.1.187', '255.255.255.0', '192.168.1.1', '192.168.1.1')
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
{'data': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01890063, 0.6546388, 2.354778, 1.975406, 0.81766, 3.085336, -6.247728, -22.59395, -16.6898, 3.000499, 16.47631, 16.58152, 4.681614, -17.70343, -18.81696, 10.21334, 16.07077, 10.53968, 13.76278, 3.499308, -21.15113, -14.28823, 15.18734, 24.60271, 22.46839, 7.021334, -10.37388, -33.92818, -7.195226, 24.81128, -3.199253, 6.644143, -9.680499, -36.34495, -0.269207, 9.124561, 2.434502, 5.526605, -2.764087, -10.13463, -4.205748, 4.149552, 8.815397, 6.750137, -6.159194, -14.55934, -2.949464, 7.381666, 5.030711, 8.313221

.{'data': [-0.007428248, 0.2725981, 2.751489, 3.4711, 0.5701023, 13.59398, 8.868208, -17.09944, 16.93612, 25.17386, -6.492106, 8.804543, 9.352472, -0.03797233, -13.52527, 10.56163, 15.27515, -30.40521, 12.98234, 26.23308, -9.576428, -4.356792, 5.478975, 14.43647, -3.725315, -15.81685, -5.407853, -5.665399, -11.1474, -4.539165, -10.35478, -12.79614, 17.83801, 17.79457, 1.934286, -2.205064, -4.531715, 4.704479, 11.45662, -8.311883, -14.02579, 3.949671, -17.19762, -1.876415, 8.452947, -15.90972, -13.44038, -12.6467, 3.524439, 5.816114, 8.913589, -12.99684, -17.56095, 7.877147, -3.214305, 9.967983, 6.483365, -9.543977, -9.261984, 2.602074, 8.67885, 8.911953, 7.345115, -3.33603, 0.02282572, 0.7434526, -0.3986041, -12.90121, -16.47241, 14.58745, 22.81265, -5.073499, -7.721611, -10.55159, -15.08033, 0.1157262, 13.83329, 16.42686, -3.799523, -9.613506, -12.01321, -12.7957, 7.914809, 11.253, 12.86936, 1.409715, -10.80211, -15.48863, -16.96552, 9.98378, 18.28639, 10.26882, 5.362011, -8.464462, -

.{'data': [0.02556564, 1.699374, 6.75367, 0.6207559, -14.57813, -4.46817, 5.411001, -1.428444, -4.217455, -0.918537, 20.09241, 7.932251, -2.261643, 1.036787, -18.93932, 6.719934, 11.81336, -1.493946, 12.34292, 11.7835, -9.886164, -13.78657, 8.366728, -3.862878, -13.78214, -4.467126, 13.20508, -9.565168, -16.47295, 7.272088, -6.864446, -0.3165381, 0.8926783, 15.984, -14.68788, -8.596444, 42.26862, -2.102741, -4.256083, 5.857686, 10.0596, -3.19337, -10.4021, 17.26972, -0.406445, -4.444401, 2.384272, 10.28656, -9.246164, 1.510052, 19.99894, 1.273005, -1.570498, -16.14552, 3.885994, -15.28683, -17.7932, 16.33588, -2.931551, -1.240051, 7.070444, 0.0457685, -12.48033, -9.471032, -0.01791241, -1.364032, -4.85031, 4.222765, 17.87402, -3.733669, -9.545906, 19.61914, 15.00811, 15.5818, 9.854746, -20.28849, -14.42008, 7.197804, 19.97007, 6.86804, -18.24351, -19.67939, 1.61303, 14.64972, 8.840837, 12.28885, 4.042869, -22.06363, -20.66054, -1.373639, 0.6200025, 11.99651, 15.80897, -5.037657, -9.495

.{'data': [-0.02122976, -0.5850621, -0.633004, 1.91552, 0.389618, 3.108453, 11.69904, -3.593241, -10.44164, 14.1091, 13.49229, -16.63488, -12.03514, 14.95601, 3.855083, -3.201916, 14.83337, 19.71491, -0.7089627, -6.23147, 1.143712, -17.88849, -4.122584, 14.06542, -9.427601, -4.177748, 15.58353, -16.67542, -14.42296, 9.470507, -6.37732, 30.02022, 6.488125, -35.19209, -1.943129, -0.3320382, 8.103146, -1.186718, 2.879054, 7.592005, -4.745014, 20.51656, -2.921368, -9.029295, 5.76592, -10.56639, -11.15448, -3.50599, -2.188162, -14.80285, -0.8537428, -2.445064, -1.4202, 17.24483, 10.65857, 10.64679, -12.74581, -11.31097, -1.794369, -6.022552, 10.88434, 7.679523, 4.157482, -6.278934, 0.02510396, 0.9067629, 1.117606, -5.474042, 0.8499985, 22.37192, 8.200348, -7.545832, -3.744581, -15.28411, 1.875438, 17.45721, 13.55387, 11.19046, -6.549663, -8.647846, -9.675999, -5.00274, 17.54816, 18.54737, 6.329535, 3.450685, -7.853503, -14.95633, 5.015184, 15.78487, 15.25507, 13.27545, -5.636652, -20.2913, 

Traceback (most recent call last):
  File "<stdin>", line 22, in <module>
KeyboardInterrupt: 


In [16]:
runner.stop()

In [62]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

p26.off()
p13.off()

print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.envh")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)
# ssid = 'vodafone749282'
# password = 'c4Eqssga2YAdAZ97'
# connect_wifi(ssid, password)

#send 7hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/7hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/7hz", toSend)

    
gc.collect()

p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 10hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/10hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/10hz", toSend) 
gc.collect()

p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 12hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/12hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/12hz", toSend) 
    

requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
# requests.GETRequest("http://192.168.1.106:5001/isCalibrated")
# requests.GETRequest("http://172.20.10.2:5001/isCalibrated") 
print(3)
# runner.run()

while True:
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"raw_data":data}
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/decode", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/decode", toSend) 
    

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.connecting to network...
....ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:5204
load:0x40078000,len:12136
load:0x40080400,len:3496
entry 0x4008063c
connecting to network...
.network config: ('172.20.10.4', '255.255.255.240', '172.20.10.1', '172.20.10.1')
ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
b'\xed21:\xb2j\n\r\n'b'rst:0x3 (SW_R\xffSET),boot:0x13 (SPI_FAST_FLASH_BOOT)\r\n'configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:5204
load:0x40078000,len:12136
load:0x40080400,len:3496
entry 0x4008063c
ets Jul 29 2019 12:21:46

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsi

In [84]:
from lib.utils import connect_wifi, load_env_vars
env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [36]:
runner.stop()

In [88]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
# ssid='Rishil'
# password='rishilhotspot'
connect_wifi(ssid, password)

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

for i in range(10):
    time.sleep(4)
    data = runner.output_buffer
    print(data)
    toSend = {"7hz":data}
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)

p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

for i in range(10):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"10hz":data}
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)
    
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

for i in range(10):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"12hz":data}
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)

network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1984075, -11.51008, -51.09663, -54.34016, -25.10506, -80.84834, -65.98896, 38.53127, 110.5019, 85.78559, -173.1855, -58.5917, 261.1672, -40.81354, -166.3448, 40.46989, -2.744927, -27.67411, -65.35244, -4.009147, 132.892, 57.52497, -84.33148, -35.17655, 199.7903, -9.632683, -203.5127, 62.37121, 107.4526, 22.0047, -95.26085, 19.46092, 164.8912, 7.063524, -45.46688, -48.59735, 127.4853, 64.51406, -199.1884, -7.238161, 129.175, 38.68983, -111.1193, -19.91751, 179.2565, 20.85125, -57.02955, -64.20248, 16.26888, 102.5544, -

Traceback (most recent call last):
  File "<stdin>", line 46, in <module>
  File "lib/requests.py", line 163, in JSONRequest
  File "lib/requests.py", line 381, in OpenRequestJSONData
  File "lib/requests.py", line 282, in OpenRequest
Exception: Error to connect to 192.168.0.13:5001


In [43]:
runner.stop()

# HOME WIFI SEND DATA

In [123]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

p26.off()
p13.off()

print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)
# ssid = 'vodafone749282'
# password = 'c4Eqssga2YAdAZ97'
# connect_wifi(ssid, password)

#send 7hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/7hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/7hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/7hz", toSend)

    
gc.collect()

p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 10hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/10hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/10hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/10hz", toSend) 
gc.collect()

p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 12hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/12hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/12hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/12hz", toSend) 
    

requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
# requests.GETRequest("http://192.168.1.106:5001/isCalibrated")
# requests.GETRequest("http://172.20.10.2:5001/isCalibrated") 
print(3)
# runner.run()

while True:
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"raw_data":data}
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/decode", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/decode", toSend) 

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.{'7': [-0.01437311, 0.05307239, 4.476276, 14.60536, 11.46655, 4.952394, 9.786938, -12.35519, -18.74934, 13.17863, 0.09847087, -4.543498, 21.9205, -14.54827, -15.08947, 20.48138, -24.5517, -22.3811, 15.0785, 7.903934, 1.840824, -17.1092, -12.67274, 21.00229, 11.55381, -28.8528, -8.043957, 16.23606, -13.51426, -20.16376, 13.75667, 24.26164, -6.463498, -6.902459, 4.934539, 13.88583, -4.629503, -25.49635, 3.466162, 7.326106, -5.437344, -15.0289, 1.880316, 15.61987, 0.1417886, -7.343251, 10.87718, 6.167267, -34.05396, 7.549204, 17.28724, 1.685781, 17.32236, -10.14492, -8.436327, 16.54729, 14.18618, -18.40067, -3.154886, 20.182, 8.307745, -5.178813, -21.00123, 2.713574, 0.005184489, 1.392382, 9.277109, 9.217062, -15.69525, -12.03453, 12.09522, -12.67041, -14.67563, 21.46524, 17.02237, -10.60517, -1.890352, 29.4941

42784
54384
.{'7': [0.02742076, 1.463178, 5.96691, 2.50098, -9.018497, -2.451346, 0.1912714, -2.164094, 6.871375, -12.21562, 0.915216, 23.16275, -12.40787, -13.53351, 11.20558, -2.958199, -7.154449, 17.23709, -3.659894, -13.78064, -5.735315, 1.352415, 31.69036, 1.502566, -24.63546, -6.227464, 11.25579, -1.490342, -3.484363, -1.152474, -15.53936, 17.79684, -11.10853, -17.26179, 20.94905, -14.7996, 11.56735, 21.76208, -17.08346, 16.63444, 17.34048, -12.77205, -3.495189, -5.2507, -12.3482, 10.04574, -1.910842, -23.82603, 8.035815, -20.2834, -6.395133, 11.62141, -25.70043, 28.68547, 5.355424, 3.301392, 24.94058, -27.84456, -3.690809, 5.15949, 6.833164, 3.264386, -8.57305, -7.424885, -0.02673973, -1.493867, -4.572511, 0.7366102, -2.168471, -19.42534, 5.345996, 15.53634, -21.76105, -7.386564, -0.2424917, -29.06485, -6.790409, 29.01207, 5.204288, -28.15759, -2.169305, 14.56355, -13.98293, -14.63463, 24.37524, 18.32394, -20.66904, 7.270948, 32.9841, -3.722889, -16.09589, -3.068809, -3.097088, 

52896
55408
.{'10': [0.02729526, 1.335737, 7.57964, 10.62558, -5.167918, -6.453134, 19.35817, 1.513555, -16.03538, 12.50994, 3.140151, 7.42783, 11.60282, -25.24805, 3.21176, 4.0741, -25.32748, 11.67128, 18.13654, -14.7098, -13.24381, 16.47429, 11.1425, 14.56645, 8.905963, -16.32623, 21.46355, 5.69145, -5.129819, 23.54567, -14.2269, 5.31778, 17.84934, -22.66333, 14.15145, 1.015639, -35.71426, 3.760345, -15.66089, 0.1004996, 21.9308, -15.62634, 15.28055, 13.54793, -8.345072, 14.15794, 19.76801, -5.172538, -13.23544, -22.49577, -17.67874, 16.56381, -20.07012, 2.744762, 13.45958, -15.91658, 25.12746, -13.36924, -9.060724, 2.111156, -15.11124, 6.766002, -27.54154, -1.598512, -0.008481208, 0.2818547, 3.121216, 8.056913, 4.622066, -9.808915, -15.87243, -8.372523, 0.6118433, -6.512994, -21.53802, 20.32074, 26.74551, -27.97873, -5.125495, -7.028803, -10.47595, 8.408497, -18.28876, 5.775237, 17.03134, -5.835138, -16.32526, 9.099936, 10.15888, -25.5805, -0.3973131, 13.95828, 8.814516, -7.031229, 

31984
55408
.{'12': [-0.01995181, -0.4664354, 1.406232, 5.41892, -6.255249, -9.098042, 10.65278, -12.33175, -22.00267, 18.70658, 14.93021, -18.08414, -8.061352, 39.85869, 16.41011, -24.18425, -2.60675, -4.475266, -10.05891, -27.57725, -17.19445, 4.201612, -5.167494, 15.42213, -11.67302, -27.69866, 0.2228026, -6.148168, 5.12321, 20.59383, -10.74587, 2.94729, 38.40443, 7.74663, 15.67773, -9.762724, -15.1565, 36.75636, 1.793544, 10.67844, -5.228201, -18.28295, 15.28266, -16.73407, 2.465089, 4.909411, -21.88439, -19.16088, -19.16634, 4.750113, 23.55007, 1.040368, -21.48563, 2.492507, 4.044223, -1.439241, 2.773454, -12.93615, 2.181113, 10.0942, 6.466834, 18.69697, 5.140042, 2.582381, -0.02613047, -1.312184, -5.264373, -2.244962, 10.84869, 9.809496, -8.010825, -12.29994, 12.09612, 18.9083, -4.499674, -5.630079, -8.259279, 6.911914, 0.9506538, -25.49819, 12.73315, 6.074555, 0.696243, -0.2328622, 0.170733, 17.08107, -39.86155, -3.026609, 37.26923, -10.11991, -6.067862, 13.52526, -4.310606, -21

Traceback (most recent call last):
  File "<stdin>", line 115, in <module>
KeyboardInterrupt: 


In [127]:
while True:
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/decode", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/decode", toSend) 
    print("LOLOLOL")
    print(toSend)
    print("=============")
    del data
    print(gc.mem_free())
    gc.collect()
    print(gc.mem_free())

{'raw_data': [0.02994025, 1.873423, 9.358912, 14.08064, 9.575802, -0.1447046, -21.28316, -9.951143, 20.08423, 1.46207, -11.95676, -9.683325, 9.775629, 23.79659, -5.558643, -14.4602, -2.536662, 9.101278, -15.58641, -20.33255, 18.66926, 6.666611, -23.81177, -13.41483, 29.46023, 11.94607, -17.65398, 1.612298, -8.105426, -17.38808, -20.59441, 8.677924, 20.12256, -19.32301, -5.439175, 9.52704, 14.5182, -3.01331, -14.62386, 11.70549, 11.16231, 16.43035, -2.851501, -0.387154, 18.50048, -8.886683, -21.21188, 4.376425, 30.79061, -14.05081, -12.762, 29.84948, 1.276602, -0.9059265, 5.303455, 2.499905, -7.202372, -23.62605, 10.09296, 15.51042, -24.17309, -18.6386, 21.28603, 14.42953, -0.01132657, -1.349587, -6.311722, 0.4881524, 19.17055, -2.291445, -28.96625, 1.890466, 13.6457, -15.74961, -16.83894, 9.143291, 16.18725, 8.394077, -7.779105, -16.15212, 25.90798, 20.70086, -27.25458, -10.71167, -0.6206718, 2.199971, 6.437795, -19.00587, -6.655993, 24.43003, 13.64919, -6.234188, 6.820336, 23.00364, 2

Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
  File "lib/requests.py", line 163, in JSONRequest
  File "lib/requests.py", line 381, in OpenRequestJSONData
  File "lib/requests.py", line 262, in OpenRequest
  File "lib/requests.py", line 445, in URL
  File "lib/requests.py", line 86, in _urlEncode
  File "lib/requests.py", line 76, in _quote
KeyboardInterrupt: 


In [141]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)
# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

p26.off()
p13.off()

print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)
# ssid = 'vodafone749282'
# password = 'c4Eqssga2YAdAZ97'
# connect_wifi(ssid, password)

#send 7hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/7hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/7hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/7hz", toSend)
    del data
    print(gc.mem_free())
    gc.collect()
    print(gc.mem_free())

p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 10hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/10hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/10hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/10hz", toSend) 
    del data
    print(gc.mem_free())
    gc.collect()
    print(gc.mem_free())


p26.off()
p13.on()

time.sleep(10)

p26.on()
p13.on()

#send 12hz data
for i in range(4):
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/12hz", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/12hz", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/12hz", toSend) 
    del data
    print(gc.mem_free())
    gc.collect()
    print(gc.mem_free())

requests.GETRequest("http://192.168.0.13:5001/isCalibrated")
# requests.GETRequest("http://192.168.1.106:5001/isCalibrated")
# requests.GETRequest("http://172.20.10.2:5001/isCalibrated") 
print(3)

while True:
    time.sleep(4)
    data = runner.output_buffer
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/decode", toSend)
#     requests.JSONRequest("http://192.168.1.106:5001/decode", toSend)
#     requests.JSONRequest("http://172.20.10.2:5001/decode", toSend) 
    del data
    print(gc.mem_free())
    gc.collect()
    print(gc.mem_free())

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
2
.connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.{'7': [-0.03197763, -2.4139, -9.915215, 0.1048362, 16.86637, -18.48365, -18.14854, 36.81688, 18.09208, -6.66615, -13.69733, -19.23075, 9.447966, 6.774131, -26.71739, -13.76444, 34.31956, 9.370063, -18.68111, -7.434063, 6.071617, 20.84275, -23.52745, -23.29808, 18.34154, 21.88511, -14.75336, -25.54428, 27.53787, -11.69514, -32.11049, 23.10801, 31.76315, -7.503179, -28.5914, 8.271986, -21.15409, -11.55906, 22.08436, -7.352715, -7.05521, -6.820935, 8.396585, 1.54793, 28.16139, 8.004306, -32.55135, 29.70105, 13.52602, 7.832779, 24.25635, 17.64175, 16.20482, -24.60613, -9.068153, 15.7573, -20.74389, -15.39896, 31.41121, 0.8632732, -24.82833, 15.89483, 0.816215, -40.70783, 0.03609233, 0.9007247, 1.400542, -3.334747, -5.358685, -13.7734, -26.57859, 13.80035, 28.76446, -25.85839, -11.73452, 30.28161, 17.4011, -8.972

.31904
55344
{'7': [0.0187995, 1.193519, 8.99839, 9.788342, -18.42043, -10.9704, 16.31796, -3.431795, 29.03276, 28.3511, -19.47041, -2.244237, -8.203734, -7.303537, -6.011889, -6.787355, 11.74078, 26.50762, -13.82825, -31.04766, 37.46746, 7.297747, 14.82228, -1.38987, -32.76381, 39.41266, -4.814229, 2.734285, 11.61121, -17.81469, 28.06252, -2.353422, -3.012983, 19.24353, -27.8133, -28.62431, 7.154094, -10.64867, 5.935356, 11.41843, -33.75694, 15.02247, -20.269, -53.94024, 16.67847, -11.24141, -7.31215, 9.872972, -3.403437, -18.31558, -13.87988, 7.982426, -17.58279, 11.91383, 9.782992, 17.91574, 20.68692, -21.7335, 29.2662, 5.606721, -21.49253, 0.3200068, -5.684675, 20.18853, -0.04038973, -2.254763, -8.837463, -9.040523, -8.219913, -0.6755888, 31.4163, 3.512167, -31.55755, 28.71125, 21.85953, -37.46813, -12.61464, 15.24653, -9.597809, -26.70227, -0.6374273, 13.19825, 5.460159, -5.361327, -6.461277, 22.0288, 4.522106, -30.07203, -10.26658, 12.892, 4.316492, -35.76665, -3.224638, 41.31959

.5456
54288
{'10': [-0.03809921, -1.933708, -5.406115, 2.320693, 4.11665, -4.515207, 11.40337, -11.28125, -16.9196, 27.73973, 0.0953989, -4.51876, -23.77206, -17.57152, 51.37582, 0.0883255, -6.891211, 8.708037, -0.4568405, 33.99071, -19.44818, -30.01689, 27.90233, -21.35294, -40.17799, 32.46295, 18.09489, 3.29902, 8.621685, -14.86637, 20.94247, 1.688162, -25.74542, 20.82364, -0.5139136, -23.60782, 12.96063, -19.21092, -17.88364, 34.89436, 11.13838, 5.688586, -7.227937, -14.45961, 9.185309, -20.34199, -19.78133, 17.9137, 1.228702, 0.4283547, 26.02586, -7.713697, -20.1769, -18.37387, -7.949628, -2.995913, -10.71364, 32.83469, 32.94538, 8.22383, -12.43714, -16.61015, -4.732484, 4.210294, -0.01525337, -1.960295, -9.755352, -1.745602, 21.6903, 6.018238, 13.33534, 26.40081, -27.57205, -16.53312, -7.789647, -13.6316, 0.2289591, -27.27516, 23.73135, 31.02342, -21.07263, -10.42328, 26.37167, 13.46803, -35.47287, -1.078293, 2.240958, -13.31441, -11.91995, 7.842946, 28.92573, -21.0712, -3.238508,

.45856
54288
{'12': [0.01405301, 0.7790339, 1.052065, -2.179982, 10.13773, 19.73112, -13.26324, -7.200187, 8.564656, -14.46802, 25.56191, 15.26283, -22.2363, 19.14166, 5.607253, 0.1649508, 22.92904, -22.91686, -15.56204, 13.16498, -31.37319, -3.56744, 10.57392, -14.72703, 26.50404, -5.284345, -5.25001, 44.51271, 2.432574, 0.8736629, 13.00629, -21.54575, -20.17937, 1.96001, 0.2317722, 4.579769, 13.95666, -23.14863, -9.085972, 15.4465, -9.77081, -19.52975, -11.31234, 30.7034, 3.714543, -28.36638, 3.753335, 15.9101, -3.101138, -21.86212, -19.00006, -25.60248, 1.222202, 13.2147, -8.475822, -6.098761, 17.31735, 3.146212, -24.0705, 2.198384, 12.65624, 20.40908, -8.550144, -4.539417, 0.0306814, 1.636043, 5.136424, -1.119315, -3.872411, 2.811309, -21.99274, -6.662921, 34.19252, -0.4147007, -1.436534, 11.53646, 6.18442, 7.926602, -21.36881, -9.354706, 13.14466, 4.650838, -26.67068, -4.050393, 33.90587, -7.970202, -13.8439, 0.7066977, 11.57351, -14.4374, -18.66599, 32.2515, 1.753574, -15.70433, 

.45856
54288
{'raw_data': [0.02861984, 2.018677, 8.181167, 0.6141644, -15.9986, 2.34836, 3.883026, -4.918327, 30.72158, -7.389461, -28.27346, 29.33864, 11.77763, -18.49755, -11.79898, 13.6371, 16.51832, 10.1381, 10.9181, 5.285375, 9.699335, -21.37377, -16.24774, 4.274244, 11.80926, 8.107477, -0.4238219, 8.052933, 4.946804, 15.59022, -8.562119, -1.43032, -6.721374, 1.868433, 13.6597, -25.05128, 24.20431, -6.357926, -16.57152, 12.69737, -27.0387, 5.061964, 16.63256, -16.32195, -17.80097, 2.421864, 8.606827, 22.11319, -12.21996, -13.8745, 14.11618, -12.54123, 12.19968, -3.00517, -8.759905, -1.624716, -28.28085, -1.689159, 23.52137, -3.47337, -13.01479, 10.9357, 2.46042, 15.37212, -0.02607402, -0.7031538, -1.345878, -0.4973017, -3.793858, 7.173556, 19.76293, -19.96746, -19.27474, 8.626422, -22.41588, 1.037643, 16.97477, -5.504378, -14.2378, -1.303271, 24.956, -18.34377, 2.027187, 26.81823, -15.67995, -16.66233, -11.70343, 7.605643, -0.0342167, -15.57299, -10.59275, -5.002722, -13.62423, -2

...

44976
55312
.{'raw_data': [-0.00241596, -0.9541949, -6.424475, -8.45138, 3.900885, 7.798916, 9.420189, 5.471642, -17.79484, 12.92016, 15.20613, -19.16227, 16.45858, -2.388647, -2.525685, 16.8345, -27.60565, 5.643054, 16.77016, 0.6915472, 14.21855, 2.105024, 7.562078, 0.9993752, -0.6596135, 4.718755, -0.1974679, 12.594, 13.78592, -10.76757, -10.0369, 2.58893, 0.8977361, 10.00533, -2.381188, -0.7630219, -6.662959, -4.161695, -12.37589, -29.75632, 6.871739, -21.81204, -7.253167, 9.800299, -11.50487, -5.508073, -15.07419, 21.43919, 6.961119, 8.44475, -2.698477, -34.22636, 14.85035, 11.67692, -18.94418, -13.62097, 19.08936, 12.92441, -13.30113, 3.682031, 21.21673, -7.801034, -15.51506, 33.23096, 0.02796957, 1.626955, 5.298535, -0.3234159, -1.370398, 15.31811, -5.873877, -18.60999, 21.37827, 12.86849, -6.396884, -13.84005, -17.33875, 15.66922, -6.275721, -25.51769, 9.939737, 16.02647, -5.186948, -11.39233, 12.3805, 16.5136, -1.861592, -11.20951, -10.48212, 1.271278, -5.425379, -14.21474, 8.9

.31856
55312
{'raw_data': [0.01272798, -0.1531543, -3.73953, -7.006873, 1.679234, -4.38098, -13.37216, 18.06123, 17.49041, -13.76107, -0.8173018, 24.45429, 5.199514, -26.71437, -1.67682, 14.37432, 1.043631, -11.18629, 1.532969, 27.11463, -9.598446, -17.4267, 3.376656, 0.2806488, -2.660925, -13.74175, -6.847215, -3.787598, 8.02618, 7.23258, -2.877061, 11.37644, -3.824092, 11.86163, 17.39628, -21.965, 4.292983, 15.94489, -11.03241, 4.290314, 15.30961, -15.23645, -3.760681, 11.6738, -8.207333, 24.47871, -10.21356, -24.87371, 20.93367, -19.17057, 8.172392, 3.463598, -34.74578, 16.17254, 4.535954, 14.78465, 15.78245, -32.03822, 0.5627928, 5.175192, -3.849028, 16.58416, -2.058161, -14.95014, 0.01713513, 0.6588635, -0.05293402, -8.213659, -6.252749, 13.24029, 9.234194, 0.08925665, -3.956605, -24.4305, 1.956583, 18.01882, -9.016382, 4.788486, -0.5240813, -1.550954, 1.184131, -16.14744, 4.387903, -10.20661, -9.831608, 11.30952, -15.64602, -19.08381, 1.613234, -14.07692, -13.83917, 8.357213, 3.2

45184
55312
.{'raw_data': [-0.02781952, -1.67967, -7.209649, -3.627732, 12.35887, 4.126034, -19.02608, -16.94232, -0.1115956, 2.587541, -5.599545, 14.99931, -5.196603, -23.90452, 33.16231, 12.69705, -13.14006, -14.32825, -16.24957, 20.77152, -5.713088, 0.9746304, -8.745257, -19.83094, 30.06267, -1.099943, 11.48874, 1.180298, -25.32005, 29.08594, 6.40905, -8.43279, -15.37418, -10.92318, 30.40912, -1.060515, 1.57682, -4.32762, -19.06801, 27.0376, 6.411376, -4.855413, 0.8751913, -6.39444, -3.34513, -6.025135, 19.20753, -4.328882, -0.2665086, 4.433348, -21.46007, 1.214844, -7.331424, -2.842416, -11.05885, -12.698, -11.83425, -1.812392, 14.85213, -17.95569, 13.45879, 15.17115, 7.33917, 18.03126, 0.01967677, 0.8556995, 1.697913, -0.1082573, 3.736033, -1.222309, -11.75704, 15.53277, 21.62187, -6.875217, -8.149388, 7.719566, 7.840682, -23.42734, -5.705301, 26.29888, 3.365122, -16.46147, 6.080087, 28.94094, -13.93344, -20.03244, 1.984167, -11.94339, -12.69054, -8.905817, 5.018748, 16.62318, 8.6

9360
55312
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 115, in <module>
KeyboardInterrupt: 


In [142]:
runner.stop()

In [140]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

[leftinbuffer] ['ets Jul 29 2019 12:21:46']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0018,len:4']
[leftinbuffer] ['load:0x3fff001c,len:5204']
[leftinbuffer] ['load:0x40078000,len:12136']
[leftinbuffer] ['load:0x40080400,len:3496']
[leftinbuffer] ['entry 0x4008063c']
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.
